In [2]:
# pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [31]:
import os
import json
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import random

# Parameters to easily change at the top
MAX_VIDEOS_TO_LIKE = 10
MAX_CHANNELS_TO_SUBSCRIBE = 10
MAX_VIDEOS_TO_COMMENT = 5
VIDEO_FILE_PATH = 'Video.mov'

# List of potential comments
COMMENTS = [
    "Great video! Very inspiring.",
    "Loved the quotes in this video!",
    "This video made my day!",
    "Amazing content, keep it up!",
    "Thanks for sharing these quotes!"
]

# Set up YouTube API client
SCOPES = [
    "https://www.googleapis.com/auth/youtube.force-ssl",
    "https://www.googleapis.com/auth/youtube.upload",
    "https://www.googleapis.com/auth/youtube",
    "https://www.googleapis.com/auth/youtubepartner",
    "https://www.googleapis.com/auth/youtube.readonly",
    "https://www.googleapis.com/auth/youtube.channel-memberships.creator",
]

CLIENT_SECRETS_FILE = "clientSecretInfo.json"  # Path to your client secret file
CREDENTIALS_FILE = 'token.json'  # File to store the user's access and refresh tokens

def get_authenticated_service():
    credentials = None
    # Check if the credentials file already exists
    if os.path.exists(CREDENTIALS_FILE):
        credentials = Credentials.from_authorized_user_file(CREDENTIALS_FILE, SCOPES)
    
    # If no valid credentials are available, let the user log in.
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_local_server(port=0)
        
        # Save the credentials for the next run
        with open(CREDENTIALS_FILE, 'w') as token:
            token.write(credentials.to_json())
    
    return build('youtube', 'v3', credentials=credentials)


# Build the YouTube API client
youtube = get_authenticated_service()

In [32]:
# Function to search for videos related to a query
# def search_videos(query, max_results):
#     request = youtube.search().list(
#         q=query,
#         part="id,snippet",
#         type="video",
#         maxResults=max_results
#     )
#     response = request.execute()  # Execute the request
#     return response['items']  # Return the list of videos

# Function to search for the most recent English videos related to a query with additional parameters
def search_videos(query, max_results):
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        order="date",  # Order by date to get the most recent videos
        maxResults=max_results,
        relevanceLanguage="en",  # Filter for English videos
        videoDuration="short",  # Filter for short and medium length videos
        regionCode="US"  # Example: Filter results to the United States
    )
    response = request.execute()
    return response['items']



In [33]:
# Function to like a list of videos
def like_videos(video_ids):
    for video_id in video_ids:
        try:
            youtube.videos().rate(rating='like', id=video_id).execute()  # Like each video by ID
            print(f"LIKING VIDEO")
        except:
            print("unable to like video, skipping")

In [34]:
# # Function to get subscribers of a channel
# def get_subscribers(channel_id, max_results):
#     request = youtube.subscriptions().list(
#         part="subscriberSnippet",
#         channelId=channel_id,
#         maxResults=max_results
#     )
#     response = request.execute()  # Execute the request
#     return response['items']  # Return the list of subscribers

In [35]:
# Identify the channel of each quote video:
def get_channel_of_videos(video_ids):
    channels = []
    for video_id in video_ids:
        request = youtube.videos().list(
            part="snippet",
            id=video_id
        )
        response = request.execute()
        channel_id = response['items'][0]['snippet']['channelId']
        channels.append(channel_id)
    return channels

In [36]:
#Identify the subscribers of each channel:
# def get_subscribers_of_channel(channel_id, max_results):
#     request = youtube.subscriptions().list(
#         part="snippet",
#         channelId=channel_id,
#         maxResults=max_results
#     )
#     response = request.execute()
#     subscriber_channels = [item['snippet']['resourceId']['channelId'] for item in response['items']]
#     return subscriber_channels



# Function to get subscribers of a channel with error handling for private or restricted channels
def get_subscribers_of_channel(channel_id, max_results):
    try:
        request = youtube.subscriptions().list(
            part="subscriberSnippet",
            channelId=channel_id,
            maxResults=max_results
        )
        response = request.execute()
        return response['items']
    except:
        print(f"Unable to access subscriber list for channel ID {channel_id}. Skipping.")
    return []

In [37]:
# # Function to subscribe to channels of subscribers
# def subscribe_to_subscribers_channels(subscriber_channel_ids):
#     for channel_id in subscriber_channel_ids:
#         youtube.subscriptions().insert(
#             part="snippet",
#             body={
#                 "snippet": {
#                     "resourceId": {
#                         "channelId": channel_id,
#                         "kind": "youtube#channel"
#                     }
#                 }
#             }
#         ).execute()

# Function to subscribe to channels of subscribers with error handling
def subscribe_to_subscribers_channels(subscriber_channels, max_results):
    for channel in subscriber_channels[:max_results]:
        try:
            youtube.subscriptions().insert(
                part="snippet",
                body={
                    "snippet": {
                        "resourceId": {
                            "channelId": channel,
                            "kind": "youtube#channel"
                        }
                    }
                }
            ).execute()
            
            print(f"SUBSCRIBING TO CHANNEL")

        except:
            print(f"An error occurred, skipping")

In [46]:
# Function to upload a new video
def upload_video(file_path, title, description, tags, category_id):
    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags,
            "categoryId": category_id
        },
        "status": {
            "privacyStatus": "public"  # Set the video to public
        }
    }
    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=MediaFileUpload(file_path, resumable=True)
    )
    response = request.execute()  # Execute the upload request
    
    print(f"UPLOADING VIDEO")

    return response

In [47]:
# Function to delete a video
def delete_video(video_id):
    try:
        youtube.videos().delete(id=video_id).execute()
        print(f"DELETING VIDEO")
    except:
        print(f"ERROR DELETING VIDEO")


In [48]:
# Function to write a comment on a video
def write_comment(video_id, comment_text):
    youtube.commentThreads().insert(
        part="snippet",
        body={
            "snippet": {
                "videoId": video_id,
                "topLevelComment": {
                    "snippet": {
                        "textOriginal": comment_text
                    }
                }
            }
        }
    ).execute()

In [50]:
if __name__ == "__main__":
    # Example usage

    # Search for videos related to "famous quotes" and like them
    videos = search_videos("famous quotes", MAX_VIDEOS_TO_LIKE)
    video_ids = [video['id']['videoId'] for video in videos]  # Extract video IDs
    like_videos(video_ids)

    #Get channels of these videos:
    channels = get_channel_of_videos(video_ids)

    #For each channel, get their subscribers and subscribe to those subscribers' channels:
    for channel_id in channels:
        subscriber_channels = get_subscribers_of_channel(channel_id, MAX_CHANNELS_TO_SUBSCRIBE)
#         subscribe_to_subscribers_channels(subscriber_channels)
#         subscriber_channels = ["CHANNEL_ID_1", "CHANNEL_ID_2", "CHANNEL_ID_3"]
        subscribe_to_subscribers_channels(subscriber_channels, MAX_CHANNELS_TO_SUBSCRIBE)


    # Comment on a few videos
    for video_id in video_ids[:MAX_VIDEOS_TO_COMMENT]:
        comment_text = random.choice(COMMENTS)  # Select a random comment
        write_comment(video_id, comment_text)

    # Upload a new video
    upload_video(VIDEO_FILE_PATH, "My New Video", "This is a description", ["tag1", "tag2"], "22")  # 22 is the category ID for People & Blogs

    # Example: Delete a video
    # delete_video("VIDEO_ID")

    # Example: Write a comment on a video
    # write_comment("VIDEO_ID", "This is a great video!")

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/search?q=famous+quotes&part=id%2Csnippet&type=video&order=date&maxResults=10&relevanceLanguage=en&videoDuration=short&regionCode=US&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">